In [1]:
import pandas as pd 
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [2]:
df_temp = pd.read_csv("https://datahub.io/core/global-temp/r/0.csv")
df_temp.head()

,Source,Year,Mean
0,gcag,1850,-0.4177
1,gcag,1851,-0.2333
2,gcag,1852,-0.2294
3,gcag,1853,-0.2704
4,gcag,1854,-0.2915


In [8]:
chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

agent_temperatura = create_pandas_dataframe_agent(
    chat, 
    df_temp,
    verbose=True,
    agent_type="tool-calling",
    allow_dangerous_code=True
)


In [9]:
resposta = agent_temperatura.invoke({"input":"Qual a temperatura media global do ultimo ano?"})
print(resposta)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Year'] == df['Year'].max()]['Mean'].values[0]"}`


1.1755A temperatura média global do último ano disponível no dataframe é de 1.1755.

> Finished chain.
{'input': 'Qual a temperatura media global do ultimo ano?', 'output': 'A temperatura média global do último ano disponível no dataframe é de 1.1755.'}


In [10]:
resposta = agent_temperatura.invoke({"input":"Qual foi o ano com maior temperatura média?"})
print(resposta)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Mean'] == df['Mean'].max()]['Year'].values[0]"}`


2024O ano com a maior temperatura média registrada foi 2024.

> Finished chain.
{'input': 'Qual foi o ano com maior temperatura média?', 'output': 'O ano com a maior temperatura média registrada foi 2024.'}


In [11]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_openai import ChatOpenAI

In [13]:
db_sakila = SQLDatabase.from_uri("sqlite:///files/sakila_master.db")

chat = ChatOpenAI(model= "gpt-3.5-turbo-0125")

agent_executor_sakila = create_sql_agent(
    chat, 
    db=db_sakila,
    agent_type="tool-calling",
    verbose=True
)

In [14]:
response = agent_executor_sakila.invoke({"input":"Quais são os filmes mais alugados?"})
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
Invoking: `sql_db_schema` with `{'table_names': 'film'}`



CREATE TABLE film (
	film_id INTEGER NOT NULL, 
	title VARCHAR(255) NOT NULL, 
	description TEXT DEFAULT NULL, 
	release_year VARCHAR(4) DEFAULT NULL, 
	language_id SMALLINT NOT NULL, 
	original_language_id SMALLINT DEFAULT NULL, 
	rental_duration SMALLINT DEFAULT 3 NOT NULL, 
	rental_rate DECIMAL(4, 2) DEFAULT (4.99) NOT NULL, 
	length SMALLINT DEFAULT NULL, 
	replacement_cost DECIMAL(5, 2) DEFAULT (19.99) NOT NULL, 
	rating VARCHAR(10) DEFAULT 'G', 
	special_features VARCHAR(100) DEFAULT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (film_id), 
	CONSTRAINT fk_film_language FOREIGN KEY(language_id) REFERENCES language (language_id), 
	CONSTRAINT fk_film_language_original FOREIGN KEY(orig

c:\programacao\automacao\agentes-Langchain\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:338: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\programacao\automacao\agentes-Langchain\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:350: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables



Invoking: `sql_db_query` with `{'query': 'SELECT title, COUNT(*) AS num_rentals FROM film JOIN inventory ON film.film_id = inventory.film_id JOIN rental ON inventory.inventory_id = rental.inventory_id GROUP BY title ORDER BY num_rentals DESC LIMIT 10'}`


[('BUCKET BROTHERHOOD', 34), ('ROCKETEER MOTHER', 33), ('SCALAWAG DUCK', 32), ('RIDGEMONT SUBMARINE', 32), ('JUGGLER HARDLY', 32), ('GRIT CLOCKWORK', 32), ('FORWARD TEMPLE', 32), ('ZORRO ARK', 31), ('WIFE TURN', 31), ('TIMBERLAND SKY', 31)]Os filmes mais alugados são:

1. BUCKET BROTHERHOOD - 34 aluguéis
2. ROCKETEER MOTHER - 33 aluguéis
3. SCALAWAG DUCK - 32 aluguéis
4. RIDGEMONT SUBMARINE - 32 aluguéis
5. JUGGLER HARDLY - 32 aluguéis
6. GRIT CLOCKWORK - 32 aluguéis
7. FORWARD TEMPLE - 32 aluguéis
8. ZORRO ARK - 31 aluguéis
9. WIFE TURN - 31 aluguéis
10. TIMBERLAND SKY - 31 aluguéis

> Finished chain.
{'input': 'Quais são os filmes mais alugados?', 'output': 'Os filmes mais alugados são:\n\n1. BUCKET BROTHERHOOD - 34 aluguéis\n2. RO

In [16]:
response = agent_executor_sakila.invoke({"input":"Quais os nomes dos atores que atuaram em mais filmes?"})
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
Invoking: `sql_db_schema` with `{'table_names': 'actor, film_actor'}`



CREATE TABLE actor (
	actor_id NUMERIC NOT NULL, 
	first_name VARCHAR(45) NOT NULL, 
	last_name VARCHAR(45) NOT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (actor_id)
)

/*
3 rows from actor table:
actor_id	first_name	last_name	last_update
1.0000000000	PENELOPE	GUINESS	2020-12-23 07:12:29
2.0000000000	NICK	WAHLBERG	2020-12-23 07:12:29
3.0000000000	ED	CHASE	2020-12-23 07:12:29
*/


CREATE TABLE film_actor (
	actor_id INTEGER NOT NULL, 
	film_id INTEGER NOT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (actor_id, film_id), 
	CONSTRAINT fk_film_actor_actor FOREIGN KEY(actor_id) REFERENCES actor (actor_id) ON UPDATE CASCADE, 
	CONSTRAINT fk_film_actor_film FOREIGN KEY(f

c:\programacao\automacao\agentes-Langchain\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:338: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\programacao\automacao\agentes-Langchain\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:350: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables



Invoking: `sql_db_query` with `{'query': 'SELECT a.first_name, a.last_name, COUNT(fa.actor_id) as movie_count FROM actor a JOIN film_actor fa ON a.actor_id = fa.actor_id GROUP BY a.first_name, a.last_name ORDER BY movie_count DESC LIMIT 10'}`


[('SUSAN', 'DAVIS', 54), ('GINA', 'DEGENERES', 42), ('WALTER', 'TORN', 41), ('MARY', 'KEITEL', 40), ('MATTHEW', 'CARREY', 39), ('SANDRA', 'KILMER', 37), ('SCARLETT', 'DAMON', 36), ('ANGELA', 'WITHERSPOON', 35), ('GROUCHO', 'DUNST', 35), ('HENRY', 'BERRY', 35)]Os atores que atuaram em mais filmes são SUSAN DAVIS, GINA DEGENERES, WALTER TORN, MARY KEITEL, MATTHEW CARREY, SANDRA KILMER, SCARLETT DAMON, ANGELA WITHERSPOON, GROUCHO DUNST e HENRY BERRY. Eles estão listados em ordem decrescente, com o número de filmes em que atuaram.

> Finished chain.
{'input': 'Quais os nomes dos atores que atuaram em mais filmes?', 'output': 'Os atores que atuaram em mais filmes são SUSAN DAVIS, GINA DEGENERES, WALTER TORN, MARY KEITEL, MATTHEW CARREY, SANDRA KILME